In [26]:
from __future__ import division      #除数可以显示为float

from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import time                 #使用time stamp
import datetime             
import matplotlib.pyplot as plt
import math


从文件中读取文件，保存为pd对象

In [27]:
Data_Total = pd.DataFrame()
Data_Total = pd.read_csv("Data_Total_pd1.csv")


注意：
因为每个股票的价格属性不同，所以要分股票进行操作

读取文件
进行数据清洗工作
1. 删除前30，后10的数据，保持数据稳定性
2. 删除价格为0的数据

增加一个pre值，显示上一天的数据


In [28]:
Stock_list = set(list(Data_Total['code']))

#增加MACD_macd_pre
Data_Total.loc[:,"MACD_macd_pre"] = None
for stock_name in Stock_list:
    temp = []
    temp = list(Data_Total.loc[Data_Total['code']==stock_name,"MACD_macd"])[1:]
    temp.append(0)
    Data_Total.loc[Data_Total['code']==stock_name,"MACD_macd_pre"] \
    = np.array(temp)

#增加:EMA_gap_pre
Data_Total.loc[:,"EMA_gap_pre"] = None
for stock_name in Stock_list:
    temp = []
    temp = list(Data_Total.loc[Data_Total['code']==stock_name,"EMA_gap"])[1:]
    temp.append(0)
    Data_Total.loc[Data_Total['code']==stock_name,"EMA_gap_pre"] \
    = np.array(temp)
    
#增加:KDJ_J_pre
Data_Total.loc[:,"KDJ_J_pre"] = None
for stock_name in Stock_list:
    temp = []
    temp = list(Data_Total.loc[Data_Total['code']==stock_name,"KDJ_J"])[1:]
    temp.append(0)
    Data_Total.loc[Data_Total['code']==stock_name,"KDJ_J_pre"] \
    = np.array(temp)


#增加:MOM_pre
Data_Total.loc[:,"MOM_pre"] = None
for stock_name in Stock_list:
    temp = []
    temp = list(Data_Total.loc[Data_Total['code']==stock_name,"MOM"])[1:]
    temp.append(0)
    Data_Total.loc[Data_Total['code']==stock_name,"MOM_pre"] \
    = np.array(temp)
    
#增加:MOM_gap_pre
Data_Total.loc[:,"MOM_gap_pre"] = None
for stock_name in Stock_list:
    temp = []
    temp = list(Data_Total.loc[Data_Total['code']==stock_name,"MOM_gap"])[1:]
    temp.append(0)
    Data_Total.loc[Data_Total['code']==stock_name,"MOM_gap_pre"] \
    = np.array(temp)
        
#增加:OBV_pre
Data_Total.loc[:,"OBV_pre"] = None
for stock_name in Stock_list:
    temp = []
    temp = list(Data_Total.loc[Data_Total['code']==stock_name,"OBV"])[1:]
    temp.append(0)
    Data_Total.loc[Data_Total['code']==stock_name,"OBV_pre"] \
    = np.array(temp)

#增加:Long_Short_Rate_OBV_pre
Data_Total.loc[:,"Long_Short_Rate_OBV_pre"] = None
for stock_name in Stock_list:
    temp = []
    temp = list(Data_Total.loc[Data_Total['code']==stock_name,"Long_Short_Rate_OBV"])[1:]
    temp.append(0)
    Data_Total.loc[Data_Total['code']==stock_name,"Long_Short_Rate_OBV_pre"] \
    = np.array(temp)
    
    
print Data_Total

      Unnamed: 0         code        date  avg_price win_rate  MACD_dif  \
0              0  300070.XSHE  2010-02-02      0.000     even  0.000000   
1              1  300070.XSHE  2010-02-03      0.000     even  0.000000   
2              2  300070.XSHE  2010-02-04      0.000     even  0.000000   
3              3  300070.XSHE  2010-02-05      0.000     even  0.000000   
4              4  300070.XSHE  2010-02-08      0.000     even  0.000000   
5              5  300070.XSHE  2010-02-09      0.000     even  0.000000   
6              6  300070.XSHE  2010-02-10      0.000     even  0.000000   
7              7  300070.XSHE  2010-02-11      0.000     even  0.000000   
8              8  300070.XSHE  2010-02-12      0.000     even  0.000000   
9              9  300070.XSHE  2010-02-22      0.000     even  0.000000   
10            10  300070.XSHE  2010-02-23      0.000     even  0.000000   
11            11  300070.XSHE  2010-02-24      0.000     even  0.000000   
12            12  300070.

In [29]:
Stock_list = set(list(Data_Total['code']))

# 删除前30，后10的数据
for stock_name in Stock_list:
    Data_Total = Data_Total.drop(Data_Total[Data_Total['code'] == stock_name].iloc[:30].index, axis=0)
    Data_Total = Data_Total.drop(Data_Total[Data_Total['code'] == stock_name].iloc[-10:].index, axis=0)
    
    
# 统一操作，删除价格为0的数据
Data_Total = Data_Total.drop(Data_Total[Data_Total['avg_price'] == 0].index, axis=0)

# 对win_rate进行操作，按照“win”为+1，“even”为0，“lose”为-1来进行
Data_Total.loc[Data_Total['win_rate'] == "win","win_rate"] = float(1)
Data_Total.loc[Data_Total['win_rate'] == "lose","win_rate"] = float(-1)
Data_Total.loc[Data_Total['win_rate'] == "even","win_rate"] = float(0)

# 更改类型为float
Data_Total[['win_rate']] = Data_Total[['win_rate']].astype(float)



归一化，用将数字归一化为-1到1之间

In [30]:
# 定义分组函数，对输入的数列按照n组进行划分
def get_group(sample_data):
    # 返回结果的数组
    ret_group = []
    
    if len(sample_data)!=0:
        # 确定最大值、最小值
        # 同时gap适当扩大，避免出现n+1的分组
        d_mean = np.mean(sample_data)
        d_std = np.std(sample_data)
        
        ret_group = [math.floor(vals) if abs(math.floor(vals)) <=3 else 4*abs(math.floor(vals))/math.floor(vals) for vals in (np.array(sample_data) - d_mean)/(0.5*d_std)]
        
    else:
        ret_group =  []
        
    return ret_group
 

开始每个股票进行数列的离散化，按照0.5个标准差进行离散

In [31]:
Data_Normalize = pd.DataFrame()

# 赋值列：code、date，avg_price，win_rate
Data_Normalize = Data_Total[['code','date','avg_price','win_rate']]

# 赋值列：MACD_macd
Data_Normalize.loc[:,"MACD_macd"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"MACD_macd"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MACD_macd"])))

# 赋值列：EMA_gap
Data_Normalize.loc[:,"EMA_gap"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"EMA_gap"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["EMA_gap"])))


# 赋值列：KDJ_J
Data_Normalize.loc[:,"KDJ_J"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"KDJ_J"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["KDJ_J"])))
    
# 赋值列：RSI
Data_Normalize.loc[:,"RSI"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"RSI"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["RSI"])))
    
# 赋值列：MOM
Data_Normalize.loc[:,"MOM"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM"])))

    
# 赋值列：MOM_gap
Data_Normalize.loc[:,"MOM_gap"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM_gap"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM_gap"])))
    
    
# 赋值列：OBV
Data_Normalize.loc[:,"OBV"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"OBV"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["OBV"])))
    

    
# 赋值列：Long_Short_Rate_OBV
Data_Normalize.loc[:,"Long_Short_Rate_OBV"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"Long_Short_Rate_OBV"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["Long_Short_Rate_OBV"])))


# KDJ_J_pre  MOM_pre  MOM_gap_pre     OBV_pre Long_Short_Rate_OBV_pre  

# 赋值列：KDJ_J_pre
Data_Normalize.loc[:,"KDJ_J_pre"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"KDJ_J_pre"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["KDJ_J_pre"])))

# 赋值列：MOM_pre
Data_Normalize.loc[:,"MOM_pre"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM_pre"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM_pre"])))

# 赋值列：MOM_gap_pre
Data_Normalize.loc[:,"MOM_gap_pre"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM_gap_pre"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM_gap_pre"])))
    
# 赋值列：OBV_pre
Data_Normalize.loc[:,"OBV_pre"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"OBV_pre"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["OBV_pre"])))

# 赋值列：Long_Short_Rate_OBV_pre
Data_Normalize.loc[:,"Long_Short_Rate_OBV_pre"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"Long_Short_Rate_OBV_pre"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["Long_Short_Rate_OBV_pre"])))

    
# MACD_macd_pre  EMA_gap_pre
# 赋值列：MACD_macd_pre
Data_Normalize.loc[:,"MACD_macd_pre"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"MACD_macd_pre"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MACD_macd_pre"])))
    
# 赋值列：EMA_gap_pre
Data_Normalize.loc[:,"EMA_gap_pre"] = None

for stock_name in Stock_list:
    Data_Normalize.loc[Data_Normalize['code']==stock_name,"EMA_gap_pre"] \
    = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["EMA_gap_pre"])))
    
print Data_Normalize.head()

           code        date  avg_price  win_rate  MACD_macd  EMA_gap  KDJ_J  \
50  300070.XSHE  2010-04-21     148.40       1.0          4        4      1   
51  300070.XSHE  2010-04-22     151.15       1.0          4        4      4   
52  300070.XSHE  2010-04-23     158.37      -1.0          4        4      4   
53  300070.XSHE  2010-04-26     170.65      -1.0          4        4      3   
54  300070.XSHE  2010-04-27     169.50      -1.0          4        4      2   

    RSI  MOM  MOM_gap  OBV  Long_Short_Rate_OBV  KDJ_J_pre  MOM_pre  \
50    4    4        4   -2                    1          4        4   
51    4    4        4   -2                    1          4        4   
52    4    4        4   -2                    1          3        4   
53    4    4        4   -2                    1          2        4   
54    4    4        4   -2                    1          1        1   

    MOM_gap_pre  OBV_pre  Long_Short_Rate_OBV_pre  MACD_macd_pre  EMA_gap_pre  
50            4   

In [32]:
print Data_Normalize
Data_Normalize.to_csv('Data_Normalize.csv')

             code        date  avg_price  win_rate  MACD_macd  EMA_gap  KDJ_J  \
50    300070.XSHE  2010-04-21    148.400       1.0          4        4      1   
51    300070.XSHE  2010-04-22    151.150       1.0          4        4      4   
52    300070.XSHE  2010-04-23    158.370      -1.0          4        4      4   
53    300070.XSHE  2010-04-26    170.650      -1.0          4        4      3   
54    300070.XSHE  2010-04-27    169.500      -1.0          4        4      2   
55    300070.XSHE  2010-04-28    157.100      -1.0          2        4      1   
56    300070.XSHE  2010-04-29    149.750      -1.0         -4        4      1   
57    300070.XSHE  2010-04-30    148.800      -1.0         -4        4      1   
58    300070.XSHE  2010-05-04    138.725      -1.0         -4        4     -2   
59    300070.XSHE  2010-05-05    140.125      -1.0         -4        4     -3   
60    300070.XSHE  2010-05-06    140.655      -1.0         -4        4     -4   
61    300070.XSHE  2010-05-0